In [2]:
import pyeda.inter as bdd
import pyeda
%load_ext gvmagic
import gvmagic 

In [53]:
class OperationCoding:
    
# Кодировние операции "or" (4 входа и два выхода)
def boolOr(x1, x2, y1, y2):
    f = x1 & y1 | x1 & ~y2 | ~x2 & y1
    g = ~x1 & x2 | ~y1 & y2
    return(f, g)

# Кодировние операции "and" (4 входа и два выхода)
def boolAnd(x1, x2, y1, y2):
    f = x1 & y1 | x1 & y2 | x2 & y1
    g = ~x1 & x2 & ~y1 & y2
    return(f, g)

# Кодировние операции "not" (2 входа и два выхода)
def boolNot(x1, x2):
    f = x1
    g = ~x1 & ~x2
    return(f, g)

# Кодировние операции "nand" (4 входа и два выхода)
def boolNand(x1, x2, y1, y2):
    return boolNot(boolAnd(x1, x2, y1, y2)[0], boolAnd(x1, x2, y1, y2)[1])

# Кодировние операции "nor" (4 входа и два выхода)
def boolNor(x1, x2, y1, y2):
    return boolNot(boolOr(x1, x2, y1, y2)[0], boolOr(x1, x2, y1, y2)[1])

# Кодировние операции "xor" (4 входа и два выхода)
def boolXor(x1, x2, y1, y2):
    f = x1 | y1
    g = ~x1 & x2 & ~y1 & ~y2 | ~x1 & ~x2 & ~y1 & y2
    return(f, g)

# Кодировние операции "xnor" (4 входа и два выхода)
def boolXnor(x1, x2, y1, y2):
    return boolNot(boolXor(x1, x2, y1, y2)[0], boolXor(x1, x2, y1, y2)[1])

# Кодировние операции "_DC" (4 входа и два выхода)
def boolDC(x1, x2, y1, y2):
    f = x1 | y1 | y2
    g = ~x1 & x2 & ~y1 & ~y2
    return(f, g)

# Кодировние операции "buf" (2 входа и один выход)
def boolBuf(x1, x2):
    f = x1
    g = x2
    return(f, g)    


# Поиск всех путей в бдд от терминальной вершины 1 
# Если необходим поиск всех путей в бдд от терминальной вершины 0, 
# то на вход необходимо подать ~f
def findAllWays(f):
    v = bdd.expr("0")
    v = bdd.expr2bdd(v)
    for i in f.satisfy_all():
        s = bdd.expr("1")
        s = bdd.expr2bdd(s)
        for a in i.items():
            if (a[1] == 1):
                s = s & a[0]
            else:
                s = s & ~a[0]
        v = v | s
    return v

# Печать ответа 
def printAnswer(r1, r2, r3, r4):
    f1 = findAllWays(~r1) & findAllWays(r3)
    f2 = findAllWays(~r1) & findAllWays(r2) & findAllWays(~r3) & findAllWays(~r4)
    f3 = findAllWays(~r1) & findAllWays(~r2) & findAllWays(~r3) & findAllWays(r4)
    f = f1 | f2 | f3
    print(bdd.bdd2expr(f))
    if (f.satisfy_one() is None):
        print("Equivalent")
    else:
        print("Not equivalent")
    for i in f.satisfy_all():
        print(i)
        

In [57]:
class EquivalentCheck:
    
with open('cases/case1/gf.v', 'r') as gf:
    text = gf.read().replace("\n","")
with open('cases/case1/rf.v', 'r') as rf:
    text1 = rf.read().replace("\n","")
    
class Parser:
    
    def __init__(self, res, left, right, operation):
        self.res = res
        self.left = left
        self.operation = operation
        self.right = right
        
# Поиск названия перменных по файлу и их инициализация
# invar - входные переменные, outvar - выходные переменные, wire - промежуточные переменные (провода)
# Возвращает словарь вида: <Название переменной в файле, фактическая переменная>
def init(text, substr, dictOfVar):
    split = text[text.find(substr) + len(substr) + 1:text.find(";",text.find(substr))].replace("\\","").replace(" ","").split(",")
    i = 0
    if substr == "input":
        inVar = bdd.exprvars("inVar", len(split) * 2)
        for varName in split:
            dictOfVar[varName] = (inVar[i], inVar[i+1])
            i += 2
    if substr == "output":
        outVar = bdd.exprvars("outVar", len(split) * 2)
        for varName in split:
            dictOfVar[varName] = (outVar[i], outVar[i+1])
            i += 2
    if substr == "wire":
        wireVar = bdd.exprvars("wireVar", len(split) * 2)
        for varName in split:
            if varName not in dictOfVar:
                dictOfVar[varName] = (wireVar[i], wireVar[i+1])
                i += 2
    return dictOfVar

def parse(text):
    dictOfVar = {}
    dictOfVar = init(text, "input", dictOfVar)
    dictOfVar = init(text, "output", dictOfVar)
    dictOfVar = init(text, "wire", dictOfVar)
    split = text[text.find(";",text.find("wire")) + 1: len(text)].replace("\\","").split(";")
    for substr in split:
        operation = substr.split(" ")[0]
        l = substr[substr.find("(") + 1: substr.find(")")].replace(" ","").split(",")
        res = l[0]
        l.remove(res)
        #print(l)
        if len(l) == 0: 
            continue
        left = l[0]
        for i in range(len(l) - 1):
            if (len(l) > 1):
                if (i >= 1):
                    left = res
                right = l[i+1]
                #print("OUT: ", res, " = ", left, " ", operation, " ", right)
            if operation == "and":
                dictOfVar[res] = boolAnd(dictOfVar[left][0], dictOfVar[left][1], dictOfVar[right][0], dictOfVar[right][1])
            elif operation == "or":
                dictOfVar[res] = boolOr(dictOfVar[left][0], dictOfVar[left][1], dictOfVar[right][0], dictOfVar[right][1])
            elif operation == "nor":
                dictOfVar[res] = boolNor(dictOfVar[left][0], dictOfVar[left][1], dictOfVar[right][0], dictOfVar[right][1])
            elif operation == "nand":
                dictOfVar[res] = boolNand(dictOfVar[left][0], dictOfVar[left][1], dictOfVar[right][0], dictOfVar[right][1])
            elif operation == "_DC":
                dictOfVar[res] = boolDC(dictOfVar[left][0], dictOfVar[left][1], dictOfVar[right][0], dictOfVar[right][1])
            elif operation == "xor":
                dictOfVar[res] = boolXor(dictOfVar[left][0], dictOfVar[left][1], dictOfVar[right][0], dictOfVar[right][1])
            elif operation == "xnor":
                dictOfVar[res] = boolXnor(dictOfVar[left][0], dictOfVar[left][1], dictOfVar[right][0], dictOfVar[right][1])
        
        #if (len(l) <= 1): print("OUT: ", res, " = ", operation, " ", left)
        if operation == "buf":
            dictOfVar[res] = boolBuf(dictOfVar[left][0], dictOfVar[left][1])
        elif operation == "not":
            dictOfVar[res] = boolNot(dictOfVar[left][0], dictOfVar[left][1])
    return dictOfVar

dictOfVar = {}
dictOfVar = init(text, "output", dictOfVar)
answ1 = execution(text)
answ2 = execution(text1)
print(dictOfVar)
for a in dictOfVar.keys():
    printAnswer(bdd.expr2bdd(answ1[a][0]), bdd.expr2bdd(answ1[a][1]), bdd.expr2bdd(answ2[a][0]), bdd.expr2bdd(answ2[a][1]))
#execution(text)
#answer()
#print(text)
        
    

{'O[19]': (outVar[0], outVar[1]), 'O[18]': (outVar[2], outVar[3]), 'O[17]': (outVar[4], outVar[5]), 'O[16]': (outVar[6], outVar[7]), 'O[15]': (outVar[8], outVar[9]), 'O[14]': (outVar[10], outVar[11]), 'O[13]': (outVar[12], outVar[13]), 'O[12]': (outVar[14], outVar[15]), 'O[11]': (outVar[16], outVar[17]), 'O[10]': (outVar[18], outVar[19]), 'O[9]': (outVar[20], outVar[21]), 'O[8]': (outVar[22], outVar[23]), 'O[7]': (outVar[24], outVar[25]), 'O[6]': (outVar[26], outVar[27]), 'O[5]': (outVar[28], outVar[29]), 'O[4]': (outVar[30], outVar[31]), 'O[3]': (outVar[32], outVar[33]), 'O[2]': (outVar[34], outVar[35]), 'O[1]': (outVar[36], outVar[37]), 'O[0]': (outVar[38], outVar[39])}
Or(And(~wireVar[1212], ~wireVar[1213], ~wireVar[1730], wireVar[1731]), And(~wireVar[1212], ~wireVar[1213], wireVar[1730]), And(~wireVar[1212], wireVar[1213], ~wireVar[1730], ~wireVar[1731]), And(~wireVar[1212], wireVar[1213], wireVar[1730]))
Not equivalent
{wireVar[1212]: 0, wireVar[1213]: 0, wireVar[1730]: 0, wireVar

In [48]:
x = bdd.bddvars('x', 2)
y = bdd.bddvars('y', 2)
z = bdd.bddvars('z', 2)

# Первая функция из примера
(_en1b1, _en1b2) = boolNand(x[0], x[1], y[0], y[1])

(_en2b1, _en2b2) = boolNot(x[0], x[1])

(_en11,_en12) = boolNot(_en1b1, _en1b2)

(_n11, _n12) = boolDC(z[0], z[1], _en1b1, _en1b2)

(_n21, _n22) = boolAnd(_n11, _n12, _en11, _en12)

(_out1, _out2) = boolDC(_n21, _n22, _en2b1, _en2b2)

# Вторая функция из примера
(en1b1, en1b2)  = boolNand(x[0], x[1], y[0], y[1])

(en2b1, en2b2) = boolNot(x[0], x[1])

(n11, n12) = boolDC(z[0], z[1], en1b1, en1b2)

(n21, n22) = boolBuf(n11, n12)

(out1, out2) = boolDC(n21, n22, en2b1, en2b2)

 
printAnswer(_out1, _out2, out1, out2)

Not equivalent
{x[0]: 0, x[1]: 1, y[0]: 0, y[1]: 0}


In [9]:
(f1, f2) = boolAnd(x[0],x[1], z[0], z[1])

(g1, g2) = boolAnd(y[0], y[1], z[0], z[1])
      
(res1, res2) = boolOr(g1, g2, f1, f2)

(f, g) = boolOr(x[0],x[1],y[0],y[1])

(res3, res4) = boolAnd(f, g, z[0], z[1])

print("f1: x & z | y & z \nf2: (x | y) & z")
printAnswer(res1, res2, res3, res4)

f1: x & z | y & z 
f2: (x | y) & z
Equivalent


In [10]:
(r1, r2) = boolAnd(x[0],x[1], z[0], z[1])

(r3, r4) = boolOr(x[0],x[1], z[0], z[1])

print("f1: x & z \nf2: x | z")
printAnswer(r1, r2, r3, r4)

f1: x & z 
f2: x | z
Not equivalent
{x[0]: 0, x[1]: 0, z[0]: 0, z[1]: 1}
{x[0]: 0, x[1]: 0, z[0]: 1}
{x[0]: 0, x[1]: 1, z[0]: 0, z[1]: 0}
{x[0]: 1, z[0]: 0, z[1]: 0}
